<a href="https://colab.research.google.com/github/smit-patel-m/SummerInternship/blob/main/PUF_XG_boost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer , classification_report, hamming_loss, accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

In [3]:
import os
from google.colab import drive

if not os.path.exists("/content/db1"):
    drive.mount('/content/drive')
    !unzip "/content/drive/MyDrive/Summer Internship/ML Attack.zip" -d "/content/db1"
else:
    print("Folder '/content/db1' already exists. Skipping unzip.")


Mounted at /content/drive
Archive:  /content/drive/MyDrive/Summer Internship/ML Attack.zip
   creating: /content/db1/ML Attack/
  inflating: /content/db1/ML Attack/Responce_Delay_2us.xlsx  
  inflating: /content/db1/__MACOSX/ML Attack/._Responce_Delay_2us.xlsx  
  inflating: /content/db1/ML Attack/Challenge.xlsx  
  inflating: /content/db1/__MACOSX/ML Attack/._Challenge.xlsx  


# Load the Excel files into pandas DataFrames

In [4]:
# Load the Excel files into pandas DataFrames

response_delay_path = "/content/db1/ML Attack/Responce_Delay_2us.xlsx"
challenge_path = "/content/db1/ML Attack/Challenge.xlsx"
response_delay_sheet_names = pd.ExcelFile(response_delay_path).sheet_names
response_delay_df = pd.read_excel(response_delay_path, sheet_name=0,header=None)

challenge_df = pd.read_excel(challenge_path,header=None)


In [5]:
# Display the first few rows of each DataFrame to verify
print("Selected Sheet Name:", response_delay_sheet_names)

print("Response Delay DataFrame:")
display(response_delay_df.head())

print("\nChallenge DataFrame:")
display(challenge_df.head())

Selected Sheet Name: ['FD_MODE_1', 'VD_MODE_1', 'FD_MODE_2', 'VD_MODE_2', 'FD_MODE_3', 'VD_MODE_3', 'FD_MODE_4', 'VD_MODE_4', 'FD_MODE_5', 'VD_MODE_5', 'FD_MODE_6', 'VD_MODE_6', 'FD_MODE_7', 'VD_MODE_7', 'FD_MODE_8', 'VD_MODE_8']
Response Delay DataFrame:


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0,1,1,1,1,0,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0
1,1,0,1,1,1,0,0,0,0,1,...,0,1,1,0,0,0,1,0,0,1
2,1,1,0,0,1,1,0,0,0,0,...,1,1,0,1,1,0,1,1,1,1
3,1,1,1,0,1,1,0,0,1,1,...,1,1,0,0,0,1,1,0,1,1
4,0,1,0,0,0,1,0,1,1,1,...,1,0,1,1,0,0,0,1,0,0



Challenge DataFrame:


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0,1,0,0,0,0,0,1,1,1,...,1,0,1,1,0,0,0,1,1,1
1,0,1,1,0,0,1,0,1,1,1,...,0,1,1,0,1,0,0,1,0,0
2,0,0,1,0,1,0,0,1,0,0,...,0,0,1,1,0,1,1,0,1,1
3,0,0,0,1,1,0,0,0,1,0,...,1,1,0,0,1,0,0,0,1,1
4,1,1,1,0,1,0,0,0,0,1,...,1,0,0,0,0,1,0,0,1,0


In [6]:
import os, re, joblib
import pandas as pd

def get_next_model_version(base_path="/content/drive/MyDrive/SummerIntern/Puf Models", prefix="puf_xgboost"):
    os.makedirs(base_path, exist_ok=True)
    existing = [f for f in os.listdir(base_path) if f.startswith(prefix)]
    versions = [int(re.findall(r'V(\d+)', name)[0]) for name in existing if re.search(r'V\d+', name)]
    next_version = max(versions) + 1 if versions else 1
    model_folder = f"{prefix}_V{next_version}"
    return os.path.join(base_path, model_folder)

def save_xgb_model(model, path, history=None):
    os.makedirs(path, exist_ok=True)

    model_path = os.path.join(path, "model.joblib")
    joblib.dump(model, model_path)
    print(f"✅ XGBoost model saved at: {model_path}")

    if history is not None:
        history_df = pd.DataFrame(history, columns=[
            "Epoch (Trees)", "Train Accuracy", "Val Accuracy",
            "Train Hamming Loss", "Val Hamming Loss", "Val Exact Match"
        ])
        excel_path = os.path.join(path, "training_history.xlsx")
        history_df.to_excel(excel_path, index=False)
        print(f" Training history saved at: {excel_path}")

def load_xgb_model(path):
    model_path = os.path.join(path, "model.joblib")
    model = joblib.load(model_path)
    print(f"✅ XGBoost model loaded from: {model_path}")
    return model


In [7]:
# Convert to NumPy
X = challenge_df.values.astype(np.float32)
Y = response_delay_df.values.astype(np.float32)


In [8]:

X_temp, X_val, Y_temp, Y_val = train_test_split(X, Y, test_size=0.10, random_state=42)
X_train, X_test, Y_train, Y_test = train_test_split(X_temp, Y_temp, test_size=2/9, random_state=42)


In [18]:
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
import numpy as np

def exact_match_accuracy(y_true, y_pred):
    return np.mean(np.all(y_true == y_pred, axis=1))

history = []

for n_estimators in range(1, 36):  # 1 to 15 trees
    xgb_base = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=None,
        learning_rate=1,
        reg_alpha=0,
        reg_lambda=0,
        use_label_encoder=False,
        eval_metric='logloss',
        tree_method='hist',
        random_state=42,
        n_jobs=-1
    )

    xgb_model = MultiOutputClassifier(xgb_base, n_jobs=-1)
    xgb_model.fit(X_train, Y_train)

    train_pred = xgb_model.predict(X_train)
    val_pred = xgb_model.predict(X_val)


    train_acc = accuracy_score(Y_train.flatten(), train_pred.flatten())
    val_acc = accuracy_score(Y_val.flatten(), val_pred.flatten())
    exact_val = exact_match_accuracy(Y_val, val_pred)

    train_hamming = hamming_loss(Y_train, train_pred)
    val_hamming = hamming_loss(Y_val, val_pred)

    print(f"\n[Epoch with {n_estimators} Trees]")
    print(f" Train Accuracy (bitwise):    {train_acc:.4f}")
    print(f" Validation Accuracy:         {val_acc:.4f}")
    print(f" Train Hamming Loss:          {train_hamming:.4f}")
    print(f" Val Hamming Loss:            {val_hamming:.4f}")
    print(f" Exact Match (Validation):    {exact_val:.4f}")

    history.append([n_estimators, train_acc, val_acc, train_hamming, val_hamming, exact_val])

    if train_acc == 1.0:
        print(f"\n✅ Achieved 100% training accuracy at {n_estimators} trees!")
        break




[Epoch with 1 Trees]
 Train Accuracy (bitwise):    0.6249
 Validation Accuracy:         0.5192
 Train Hamming Loss:          0.3751
 Val Hamming Loss:            0.4808
 Exact Match (Validation):    0.0000

[Epoch with 2 Trees]
 Train Accuracy (bitwise):    0.6674
 Validation Accuracy:         0.5162
 Train Hamming Loss:          0.3326
 Val Hamming Loss:            0.4838
 Exact Match (Validation):    0.0000

[Epoch with 3 Trees]
 Train Accuracy (bitwise):    0.7030
 Validation Accuracy:         0.5140
 Train Hamming Loss:          0.2970
 Val Hamming Loss:            0.4860
 Exact Match (Validation):    0.0000

[Epoch with 4 Trees]
 Train Accuracy (bitwise):    0.7326
 Validation Accuracy:         0.5117
 Train Hamming Loss:          0.2674
 Val Hamming Loss:            0.4883
 Exact Match (Validation):    0.0000

[Epoch with 5 Trees]
 Train Accuracy (bitwise):    0.7576
 Validation Accuracy:         0.5132
 Train Hamming Loss:          0.2424
 Val Hamming Loss:            0.4868
 E

In [19]:
# Save model + history
save_path = get_next_model_version(
    base_path="/content/drive/MyDrive/SummerIntern/Puf Models",
    prefix="puf_xgboost"
)

save_xgb_model(xgb_model, save_path, history=history)

✅ XGBoost model saved at: /content/drive/MyDrive/SummerIntern/Puf Models/puf_xgboost_V2/model.joblib
📊 Training history saved at: /content/drive/MyDrive/SummerIntern/Puf Models/puf_xgboost_V2/training_history.xlsx


In [20]:
load_path = "/content/drive/MyDrive/SummerIntern/Puf Models/puf_xgboost_V2"
xgb_model = load_xgb_model(load_path)

✅ XGBoost model loaded from: /content/drive/MyDrive/SummerIntern/Puf Models/puf_xgboost_V2/model.joblib


In [32]:
xgb_pred = xgb_model.predict(X_test)
acc =  1 - hamming_loss(Y_test, xgb_pred)
print(f"Test Accuracy : {acc:.4f}")

Test Accuracy : 0.5097


In [13]:
'''
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

plt.figure(figsize=(10, 5))
plt.plot(trees_used, train_accs, label='Train Accuracy')
plt.plot(trees_used, val_accs, label='Validation Accuracy')
plt.xlabel("Number of Trees")
plt.ylabel("Accuracy")
plt.title("XGBoost MultiOutput Accuracy vs Trees")
plt.legend()
plt.grid(True)
plt.show()
'''

' \nimport matplotlib.pyplot as plt\nfrom sklearn.tree import plot_tree\n\nplt.figure(figsize=(10, 5))\nplt.plot(trees_used, train_accs, label=\'Train Accuracy\')\nplt.plot(trees_used, val_accs, label=\'Validation Accuracy\')\nplt.xlabel("Number of Trees")\nplt.ylabel("Accuracy")\nplt.title("XGBoost MultiOutput Accuracy vs Trees")\nplt.legend()\nplt.grid(True)\nplt.show()\n'

In [14]:
xgb_train_pred = xgb_model.predict(X_train)
xgb_test_pred = xgb_model.predict(X_test)

print("\n XGBoost Evaluation")
print(f" Train Accuracy:      {accuracy_score(Y_train, xgb_train_pred):.4f}")
print(f" Train F1 Score:      {f1_score(Y_train, xgb_train_pred, average='macro'):.4f}")
print(f" Train Hamming Loss:  {hamming_loss(Y_train, xgb_train_pred):.4f}")

print(f"\n Test Accuracy:       {accuracy_score(Y_test, xgb_test_pred):.4f}")
print(f" Test F1 Score:       {f1_score(Y_test, xgb_test_pred, average='macro'):.4f}")
print(f" Test Hamming Loss:   {hamming_loss(Y_test, xgb_test_pred):.4f}")


 XGBoost Evaluation
 Train Accuracy:      0.9763
 Train F1 Score:      0.9993
 Train Hamming Loss:  0.0008

 Test Accuracy:       0.0000
 Test F1 Score:       0.5650
 Test Hamming Loss:   0.4898


In [15]:
def exact_match_accuracy(y_true, y_pred):
    return np.mean(np.all(y_true == y_pred, axis=1))

xgb_test_pred = xgb_model.predict(X_test)
print("XGBoost Exact Match Test Accuracy:", exact_match_accuracy(Y_test, xgb_test_pred))


XGBoost Exact Match Test Accuracy: 0.0


In [16]:


# Take first 5 test samples
X_train_sample = X_train[:5]
Y_train_sample = Y_train[:5]


xgb_pred_sample = xgb_model.predict(X_train_sample)

xgb_bit_errors = np.sum(xgb_pred_sample != Y_train_sample, axis=1)

print("\nBit Error Count on first 5 test samples:\n")

for i in range(5):
    print(f"Sample {i+1}:")
    print(f"  Input Challenge     : {np.round(X_train_sample[i], 2)}")
    print(f"  Target Response     : {Y_train_sample[i].astype(int)}")
    print(f"  Predicted Response  : {xgb_pred_sample[i].astype(int)}")
    print(f"  XGBoost Bit Errors  : {xgb_bit_errors[i]}")
    print("-" * 60)



Bit Error Count on first 5 test samples:

Sample 1:
  Input Challenge     : [1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0.
 0. 0. 1. 1. 0. 0. 0. 1.]
  Target Response     : [1 0 0 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0]
  Predicted Response  : [1 0 0 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0]
  XGBoost Bit Errors  : 0
------------------------------------------------------------
Sample 2:
  Input Challenge     : [0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0.
 1. 0. 1. 0. 0. 0. 0. 1.]
  Target Response     : [1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 1 0 1 1 0]
  Predicted Response  : [1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 1 0 1 1 0]
  XGBoost Bit Errors  : 0
------------------------------------------------------------
Sample 3:
  Input Challenge     : [1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 0.]
  Target Response     : 

In [17]:


# Take first 5 test samples
X_test_sample = X_test[:5]
Y_test_sample = Y_test[:5]


xgb_pred_sample = xgb_model.predict(X_test_sample)


xgb_bit_errors = np.sum(xgb_pred_sample != Y_test_sample, axis=1)

print("\nBit Error Count on first 5 test samples:\n")


for i in range(5):
    print(f"Sample {i+1}:")
    print(f"  Input Challenge     : {np.round(X_train_sample[i], 2)}")
    print(f"  Target Response     : {Y_train_sample[i].astype(int)}")
    print(f"  Predicted Response  : {xgb_pred_sample[i].astype(int)}")
    print(f"  XGBoost Bit Errors  : {xgb_bit_errors[i]}")
    print("-" * 60)



Bit Error Count on first 5 test samples:

Sample 1:
  Input Challenge     : [1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0.
 0. 0. 1. 1. 0. 0. 0. 1.]
  Target Response     : [1 0 0 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0]
  Predicted Response  : [1 0 0 0 1 0 0 0 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 0 0 1 0 1 0 1 0 1]
  XGBoost Bit Errors  : 13
------------------------------------------------------------
Sample 2:
  Input Challenge     : [0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0.
 1. 0. 1. 0. 0. 0. 0. 1.]
  Target Response     : [1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 1 0 1 1 0]
  Predicted Response  : [1 1 1 0 1 0 1 0 1 0 1 0 0 0 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 0 0]
  XGBoost Bit Errors  : 14
------------------------------------------------------------
Sample 3:
  Input Challenge     : [1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 0.]
  Target Response     